In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from os import path

In [21]:
from models.linearclassifier import LinearClassifier
from models.lstmclassifier import LSTMClassifier
from voicedataset import VoiceDataset

In [4]:
# Define data directory locations
BASE_DIR = 'dataset/Common1/'

train_male = path.join(BASE_DIR, 'train_male')
train_female = path.join(BASE_DIR, 'train_female')
test_male = path.join(BASE_DIR, 'test_male')
test_female = path.join(BASE_DIR, 'test_female')

In [6]:
# Take severals minutes
train_dataset = VoiceDataset(train_male, train_female)
test_dataset = VoiceDataset(test_male, test_female)

Done for males
Done for females
Done for males
Done for females


In [7]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [20]:
# Training Linear
model = LinearClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 10

for epoch in range(n_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 0.0152
Epoch [2/5], Loss: 0.0328
Epoch [3/5], Loss: 0.0065
Epoch [4/5], Loss: 0.0003
Epoch [5/5], Loss: 0.1783


In [18]:
model = LSTMClassifier()
criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30  

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X = batch_X.unsqueeze(1)
        batch_y = batch_y.float()
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)  # Squeeze to match the shape
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}')

Epoch [1/30], Loss: 0.1774
Epoch [2/30], Loss: 0.0888
Epoch [3/30], Loss: 0.0794
Epoch [4/30], Loss: 0.0743
Epoch [5/30], Loss: 0.0712
Epoch [6/30], Loss: 0.0669
Epoch [7/30], Loss: 0.0633
Epoch [8/30], Loss: 0.0590
Epoch [9/30], Loss: 0.0549
Epoch [10/30], Loss: 0.0537
Epoch [11/30], Loss: 0.0518
Epoch [12/30], Loss: 0.0520
Epoch [13/30], Loss: 0.0510
Epoch [14/30], Loss: 0.0502
Epoch [15/30], Loss: 0.0485
Epoch [16/30], Loss: 0.0475
Epoch [17/30], Loss: 0.0450
Epoch [18/30], Loss: 0.0441
Epoch [19/30], Loss: 0.0437
Epoch [20/30], Loss: 0.0416
Epoch [21/30], Loss: 0.0432
Epoch [22/30], Loss: 0.0406
Epoch [23/30], Loss: 0.0396
Epoch [24/30], Loss: 0.0387
Epoch [25/30], Loss: 0.0406
Epoch [26/30], Loss: 0.0392
Epoch [27/30], Loss: 0.0387
Epoch [28/30], Loss: 0.0362
Epoch [29/30], Loss: 0.0363
Epoch [30/30], Loss: 0.0353


In [20]:
# Test the model
with torch.no_grad():
    model.eval()
    correct = 0
    total = 0
    for inputs, labels in DataLoader(test_dataset, batch_size=16):
        inputs = inputs.unsqueeze(1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')

Accuracy of the model on the test set: 89.08%
